In [1]:
import numpy as np
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
class MyANNNet(nn.Module):
    def __init__(self, inputs, out1, out2, class_num):
        super(MyANNNet, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(inputs, out1),
            nn.BatchNorm1d(out1),   # 添加批标准化
            nn.ReLU(inplace=True)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(out1, out2),
            nn.BatchNorm1d(out2),
            nn.ReLU(inplace=True)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(out2, class_num)
        )
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

    
def accu(out, y):
    _, pred = torch.max(out, 1)  # 最大概率，对应的类别
    corr_num = pred.eq(y).sum()  # tensor 得取出结果
    acc = corr_num.item() / y.shape[0]
    return acc


# 单epoch的训练
def train(model, device, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for x, y in train_loader:
        x = x.view(x.size(0), -1)    # reshape(x.size(0), 该维度压平)
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        out = model(x)               # 各个分类的概率
        loss = criterion(out, y)
        train_loss += loss
        acc = accu(out, y)
        train_acc += acc
        loss.backward()
        optimizer.step()
    return train_loss / len(train_loader), train_acc / len(train_loader) 

# 测试
def test(model, device, test_loader, criterion, optimizer):
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    for x, y in test_loader:
        x = x.view(x.size(0), -1)    # reshape(x.size(0), 该维度压平)
        x = x.to(device)
        y = y.to(device)
        out = model(x)               # 各个分类的概率
        loss = criterion(out, y)
        test_loss += loss
        acc = accu(out, y)
        test_acc += acc
        loss.backward()
    return test_loss/len(test_loader), test_acc/len(test_loader)

## 接下来做什么？
- 数据标准化：torchvision.transforms 它提供了很多处理图像的方法：
    - transforms.ToTensor()将图片对象转换成pytorch中的Tensor，转换过程中自动标准化，即Tensor的范围为[0,1];
    - transforms.Normalize()需要传入两个数，第一个是均值，第二个是方差。公式：减均值，除方差。

In [15]:
batch_size = 32
lr = 1e-3
epochs = 10

In [16]:
trans_data = transforms.Compose([  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

- transforms.Compose([])    将各种操作组在一起
- 三通道transforms.Normalize([a,b,c], [d,e,f])  应该把3个通道的均值方差填入。

## 下载并读取数据集

In [5]:
train_datasets = datasets.MNIST(root='./data', train=True, transform=trans_data, download=True)
test_datasets = datasets.MNIST(root='./data', train=False, transform=trans_data, download=True)

In [6]:
n_train = int(len(train_datasets)*0.9)
n_valid = len(train_datasets) - n_train
train_datasets, valid_datasets = torch.utils.data.random_split(train_datasets, [n_train, n_valid])

In [7]:
# 生成迭代器，传入数据集，batch size，是否shuffle
train_loader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_datasets, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_datasets, batch_size=batch_size, shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyANNNet(28*28, 300, 50, 10)
model = model.to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

## 开始训练

In [12]:
model_path = './models/ann-minst.pth'
info = 'Epoch:{0}|Train Loss:{1}|Train Acc:{2}|Val Loss:{3}|Val Acc:{4}'
for epoch in range(epochs):
    train_loss, train_acc = train(model, device, train_loader, criterion, optimizer)
    valid_loss, valid_acc = test(model, device, valid_loader, criterion, optimizer)
    if valid_loss < float("inf"):
        torch.save(model.state_dict(), model_path)
        print("Save the best models to {}".format(model_path))
    print(info.format(epoch+1, train_loss, train_acc, valid_loss, valid_acc))

Save the best models to ./models/ann-minst.pth
Epoch:1|Train Loss:0.036617957055568695|Train Acc:0.9900770142180095|Val Loss:0.06657259166240692|Val Acc:0.9793882978723404
Save the best models to ./models/ann-minst.pth
Epoch:2|Train Loss:0.03458032384514809|Train Acc:0.9908360485781991|Val Loss:0.06505784392356873|Val Acc:0.980219414893617
Save the best models to ./models/ann-minst.pth
Epoch:3|Train Loss:0.03030979260802269|Train Acc:0.9918542654028436|Val Loss:0.0708872526884079|Val Acc:0.9782247340425532
Save the best models to ./models/ann-minst.pth
Epoch:4|Train Loss:0.02756541781127453|Train Acc:0.9928354561611374|Val Loss:0.06922823190689087|Val Acc:0.980219414893617
Save the best models to ./models/ann-minst.pth
Epoch:5|Train Loss:0.02680644951760769|Train Acc:0.9929650473933649|Val Loss:0.06804633140563965|Val Acc:0.9793882978723404
Save the best models to ./models/ann-minst.pth
Epoch:6|Train Loss:0.02279539406299591|Train Acc:0.993853672985782|Val Loss:0.0657816082239151|Val A

In [13]:
test(model, device, test_loader, criterion, optimizer)

(tensor(0.0600, device='cuda:0', grad_fn=<DivBackward0>), 0.9819289137380192)